###准备阶段

In [ ]:
#导入包
import time
from random import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import json
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import csv

In [ ]:
#提取文本函数
def getinfor(text):
    soup = BeautifulSoup(text, "html5lib")
    list1s = soup.find("div", class_ = "list-container").find_all("div", class_ = "default-list-item clearfix")
    ans_lst = []
    for list1 in list1s:
        item = []
        list2 = list1.find("div", class_ = "list-item-desc-top")
        a = list2.find("a")
        item.append(a.text)
        #print(a.text)
        js = json.loads(a["data-lab"])
        #print("id:{}".format(js["poi_id"]))
        item.append(js["poi_id"])
        #print(list2.find("div", class_ = "item-site-info clearfix").text)
        item.append(list2.find("div", class_ = "item-site-info clearfix").text)
        #print(list2.find("div", class_ = "item-eval-info clearfix").text)
        item.append(list2.find("div", class_ = "item-eval-info clearfix").text)
        ans_lst.append(item)
    return ans_lst

In [ ]:
#数据爬取函数
def getInformation(sector_city, ans):

    #使用无界面打开模式
    #获取美团网页源代码
    chrome_options = webdriver.ChromeOptions()
    #设置web_driver选项
    chrome_options.add_argument("--headless")
    #添加参数为无界面模式
    chrome_options.add_argument("--disable-gpu")
    #禁用GPU加速
    chrome_options.add_argument("user-agent = 'user-agent=‘Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'")
    #浏览器头
    browser = webdriver.Chrome(chrome_options=chrome_options)
    url = r"https://alaer.meituan.com/"
    browser.get(url)
    #设置网页访问cookie
    #需要更换的cookie
    cookies_txt = '''
[
{
    "domain": ".meituan.com",
    "expirationDate": 1615628583,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_lx_utm",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "utm_source%3Dbing%26utm_medium%3Dorganic",
    "id": 1
},
{
    "domain": ".meituan.com",
    "expirationDate": 1709623637,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_lxsdk_cuid",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "176b6cb42fbc8-0ccfee2da3db6-c791039-144000-176b6cb42fbc8",
    "id": 2
},
{
    "domain": ".meituan.com",
    "expirationDate": 1615026818,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_lxsdk_s",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "17806eb3fc1-6b7-ae2-f6e%7C%7C9",
    "id": 3
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": false,
    "name": "firstTime",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "1615025019039",
    "id": 4
},
{
    "domain": ".meituan.com",
    "expirationDate": 1616234618.454177,
    "hostOnly": false,
    "httpOnly": true,
    "name": "lsu",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "",
    "id": 5
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": true,
    "name": "lt",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "0NaAto8R8fpqPtXX67R2zFkxO1cAAAAA6gwAADPjOeMK_3PvQBqQRUejONDZETa7fuwioGyRob6MdvjkFVMHSuIgDVHqZo7Ey6wVXQ",
    "id": 6
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": true,
    "name": "mt_c_token",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "0NaAto8R8fpqPtXX67R2zFkxO1cAAAAA6gwAADPjOeMK_3PvQBqQRUejONDZETa7fuwioGyRob6MdvjkFVMHSuIgDVHqZo7Ey6wVXQ",
    "id": 7
},
{
    "domain": ".meituan.com",
    "expirationDate": 1615620439,
    "hostOnly": false,
    "httpOnly": false,
    "name": "mtcdn",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "K",
    "id": 8
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": true,
    "name": "n",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "mpy558877623",
    "id": 9
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": true,
    "name": "token",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "0NaAto8R8fpqPtXX67R2zFkxO1cAAAAA6gwAADPjOeMK_3PvQBqQRUejONDZETa7fuwioGyRob6MdvjkFVMHSuIgDVHqZo7Ey6wVXQ",
    "id": 10
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": true,
    "name": "token2",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "0NaAto8R8fpqPtXX67R2zFkxO1cAAAAA6gwAADPjOeMK_3PvQBqQRUejONDZETa7fuwioGyRob6MdvjkFVMHSuIgDVHqZo7Ey6wVXQ",
    "id": 11
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": false,
    "name": "u",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "2656359798",
    "id": 12
},
{
    "domain": ".meituan.com",
    "hostOnly": false,
    "httpOnly": false,
    "name": "unc",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "mpy558877623",
    "id": 13
},
{
    "domain": ".meituan.com",
    "expirationDate": 1615961717.025913,
    "hostOnly": false,
    "httpOnly": false,
    "name": "uuid",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "337e94e36f5e4c9db2b8.1615015635.1.0.0",
    "id": 14
},
{
    "domain": ".www.meituan.com",
    "expirationDate": 1677233019,
    "hostOnly": false,
    "httpOnly": false,
    "name": "__mta",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": false,
    "storeId": "0",
    "value": "209131053.1615017371078.1615025009492.1615025019125.5",
    "id": 15
}
]'''
    cookies = json.loads(cookies_txt)
    for cookie in cookies:
        cookie.pop("secure")
        cookie.pop("sameSite")

    browser.delete_all_cookies()

    for item in cookies:
        browser.add_cookie(item)

    click = browser.find_element_by_css_selector("#main > header > div.header-bar > div > div > a")
    time.sleep(random()+1)
    ActionChains(browser).double_click(click).perform()
    #点击切换城市标志
    city = browser.find_element_by_css_selector(sector_city)
    time.sleep(random()*3)
    ActionChains(browser).double_click(city).perform()
    #切换城市
    input = browser.find_element_by_css_selector(".header-search-input")
    #定位搜索输入框
    input.send_keys("美食")
    time.sleep(random()*5)
    #输入搜索信息
    browser.find_element_by_css_selector(".header-search-btn").click()
    #print(browser.get_cookies())
    text = browser.page_source
    ans.append(getinfor(text))
    time.sleep(random()*5)
    print("第一页完成")
    #打印源码
    #pages = browser.find_element_by_class_name(".clearfix")
    #print("pages:", pages.text)
    '''wrong = False

    for i in range(2, 9):
        try:
            page = browser.find_element_by_css_selector("#react > div > div > div.center-content.clearfix > div.left-content > nav > ul > li:nth-child({})".format(i))
        except NoSuchElementException:
            print("第{}页没有内容了".format(i))
            wrong = True
            break
        #print(page.text)
        ActionChains(browser).double_click(page).perform()
        #点击下一页
        time.sleep(random()*6+2)
        text = browser.page_source
        ans.append(getinfor(text))
        print("第{}页完成".format(i))
        time.sleep(random()*3+1)
    i = 8
    while wrong == False and i <= 36:
        try:
            page = browser.find_element_by_css_selector("#react > div > div > div.center-content.clearfix > div.left-content > nav > ul > li:nth-child(7)")
        except:
            break
        ActionChains(browser).double_click(page).perform()
        i += 1
        time.sleep(random()*6+2)
        text = browser.page_source
        ans.append(getinfor(text))
        print("爬取完成第{}页".format(i))
        time.sleep(random()*3+1)
        '''
    for i in range(0, 34):
        try:
            page = browser.find_element_by_css_selector("#react > div > div > div.center-content.clearfix > div.left-content > nav > ul > li.pagination-item.next-btn.active > a")
            ActionChains(browser).double_click(page).perform()
        except:
            break
        time.sleep(random()*6+5)
        text = browser.page_source
        ans.append(getinfor(text))
        print("爬取完成第{}页".format(i+2))
        time.sleep(random()*3+5)
    print("爬取完成")
    browser.quit()
    return


In [ ]:
#将爬取到的数据写入表格文件
def loadDataToCsv(data, city_name):
    #args:data,读取到的数据文件，为np.array类型
    with open(r"{}.csv".format(city_name), "w") as file:
        writer = csv.writer(file)
        writer.writerow(["餐馆名", "id", "类型", "地址", "评分", "评论数"])
        for row in data:
            for item in row:#将数据进行分割写入
                name = item[0]
                id = item[1]
                s_lst = item[2].split("|")
                #对类型+地址字符串进行切分
                if "|" in item[2]:
                #该字符串可以切分
                    kind = s_lst[0]
                    address = s_lst[1].strip("查看地图")
                else:
                    kind = " "
                    address = " "
                s_lst = item[3].split("分")
                #对评分+评论数字符串进行切分
                if "分" in item[3]:
                #该字符串可以切分
                    score = s_lst[0] + "分"
                    command_num = s_lst[1].strip("人评论")
                else:
                    score = " "
                    command_num = " "
                try:
                    writer.writerow([name, id, kind, address, score, command_num])
                except UnicodeEncodeError:
                    continue
        print("写入完成")
        file.close()


###爬取阶段

In [ ]:
#爬取数据块
ans = []
getInformation("#city-L > span.cities > a:nth-child(16)", ans)
#此处字符串参数需更换
ans_array = np.array(ans)
np.save("数据文件.npy", ans_array)

In [ ]:
#数据写入文件块
loadDataToCsv(ans_array, "来宾")
#此处文件名需更换